In [1]:
import numpy as np
import os
import fileinput
import sys
import pandas as pd
import matplotlib.pyplot as plt

from turborans.utilities.foam_automation.caserunner import run_until_convergence, get_endtime
from turborans.utilities.foam_automation.computeFoamLossFunc import computeFoamFieldLossFunc
from turborans.utilities.foam_automation.set_foam_coeff import set_foam_coef
from turborans.utilities.json_io import write_suggestion, write_json, load_coeff_default, load_coeff_bounds, load_history_loss_log, load_suggestion
from turborans.utilities.control import initialize
from turborans.update_history import register_score
from turborans.suggest_coeffs import suggest

import random

# Bayesian optimization

In [4]:
# Run simulation loop
foamdir = 'airfoil_tuner'
savedir = 'airfoil_tuner_opt'

tunerdir = os.path.join(foamdir,'tuner')
iterations=15
n_points_sampled = 4
utility = 'poi'
xi = 0.1
ref_df = pd.read_csv(os.path.join(tunerdir,'refdata.csv'))


initialize(directory = tunerdir,
           coeff_bounds = None,
           coeff_default = None,
           restart = True)

foamLoss = computeFoamFieldLossFunc(foamdir=foamdir,
                                    ref_df=ref_df,
                                    interp_method='nearest',
                                    read_cell_centres=False)
np.random.seed(7)
a1_rand = np.random.uniform(.248, .372, size=(n_points_sampled))
betaStar_rand = np.random.uniform(.072, .108, size=(n_points_sampled))
score_best = 1E6 

for i in range(iterations):
    if i == 0:
        search_point=load_coeff_default(directory=tunerdir)
        write_suggestion(directory=tunerdir, suggestion=search_point)
    elif i < n_points_sampled:
        search_point = {"a1": a1_rand[i], "betaStar": betaStar_rand[i]}
        write_suggestion(directory=tunerdir, suggestion=search_point)
    else:
        suggest(directory=tunerdir, random_state=7,utility_kind = utility, xi=xi)
    
    search_point = load_suggestion(directory=tunerdir)
    print(search_point)
    set_foam_coef(foamdir,search_point)
    last_time = run_until_convergence(case=foamdir, n_proc = 8)
    score = -foamLoss.foam_gedcp(foamtime=last_time,
                                                   coef_default_dict = load_coeff_default(directory=tunerdir),
                                                   coef_dict=search_point,
                                                   error_calc_fields=[],
                                                   error_calc_intparams=['Cl'],
                                                   error_type = 'mape'
                                                  )
                                                   
    register_score(score=score, directory=tunerdir)
    if abs(score) < abs(score_best):   
        score_best = score
        os.system(f'rm -r {savedir}')
        os.system(f'cp -r {foamdir} {savedir}')

    

INFO:general:Starting: decomposePar -force -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8169


{'a1': 0.31, 'betaStar': 0.09}


INFO:general:Finished
INFO:general:Starting: mpirun -n 8 simpleFoam -parallel -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8174
INFO:general:Finished
INFO:general:Starting: reconstructPar -latestTime -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8209
INFO:general:Finished
rm: cannot remove 'airfoil_tuner_opt': No such file or directory
INFO:general:Starting: decomposePar -force -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8220


{'a1': 0.3447099302377742, 'betaStar': 0.09138585133477561}


INFO:general:Finished
INFO:general:Starting: mpirun -n 8 simpleFoam -parallel -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8225
INFO:general:Finished
INFO:general:Starting: reconstructPar -latestTime -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8260
INFO:general:Finished
INFO:general:Starting: decomposePar -force -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8267


...

INFO:general:Finished
INFO:general:Starting: mpirun -n 8 simpleFoam -parallel -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8767
INFO:general:Finished
INFO:general:Starting: reconstructPar -latestTime -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8803
INFO:general:Finished
INFO:general:Starting: decomposePar -force -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8810


{'a1': 0.3012715615450442, 'betaStar': 0.10790235289381722}


INFO:general:Finished
INFO:general:Starting: mpirun -n 8 simpleFoam -parallel -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8815
INFO:general:Finished
INFO:general:Starting: reconstructPar -latestTime -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8853
INFO:general:Finished
INFO:general:Starting: decomposePar -force -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8860


{'a1': 0.260050585342527, 'betaStar': 0.08742859575390859}


INFO:general:Finished
INFO:general:Starting: mpirun -n 8 simpleFoam -parallel -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8865
INFO:general:Finished
INFO:general:Starting: reconstructPar -latestTime -case airfoil_tuner in /home/ryley/ML/turborans/examples/openfoam_airfoil
INFO:general:Started with PID 8902
INFO:general:Finished
